In [1]:
import warnings
warnings.filterwarnings('ignore')

Stat/CS 5525: Homework 2

Author:      Kazi Hasan Ibn Arif

Student ID:  906614469

Email:       hasanarif@vt.edu

Date:        Sep25, 2023

### (a)
Fit a linear model

In [2]:
import numpy as np
from scipy.stats import t

# Load data
data_array = np.loadtxt('Grocery.txt', delimiter='\t')
Y = data_array[:, -1] # last column
X = data_array[:, 1:-1] # X1, X2, X3

# Add a constant term for beta_0
X = np.column_stack([np.ones(X.shape[0]), X])

# Calculate coefficients using the normal equation
beta_hat = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(Y)
print("Coefficients:", beta_hat)

# Calculate residuals
residuals = Y - X.dot(beta_hat)

# Calculate residual sum of squares
RSS = residuals.T.dot(residuals)

n, k = X.shape  # (number of observations, number of variables)
sigma_hat_squared = RSS / (n - k)
print("Residual Variance (σ^2) estimate:", sigma_hat_squared)

# Calculate the standard errors for the coefficients
var_beta_hat = np.linalg.inv(X.T.dot(X)) * sigma_hat_squared
se_beta_hat = np.sqrt(np.diag(var_beta_hat))
print("Standard Errors:", se_beta_hat)

# Calculate the z-scores 
z_scores = beta_hat / se_beta_hat
print("z-scores:", z_scores)

# Calculate the p-values
p_values = (1 - t.cdf(np.abs(z_scores), df=n - k)) * 2
print("p-values:", p_values)

Coefficients: [-2.47626389e-01  2.11823566e-07  4.05522488e-02]
Residual Variance (σ^2) estimate: 0.10678636329362369
Standard Errors: [4.44595714e-01 8.30840490e-07 5.23427270e-02]
z-scores: [-0.55696981  0.25495094  0.77474467]
p-values: [0.58008333 0.79982804 0.44221318]


### (b)
Use both best subset C_p, forward variable selection, and backward variable se-
lection to find a smaller model with the best fits. Show the results of the fits as
in (a)

In [3]:
import statsmodels.api as sm
import itertools


# Fit the full model to get σ^2 (residual variance)
full_model = sm.OLS(Y, X).fit()
sigma2 = full_model.mse_resid

# Cp criterion function
def Cp(model, sigma2, n):
    RSS = sum(model.resid ** 2)
    p = len(model.params) 
    return (1/n) * (RSS + 2 * p * sigma2)

# Number of observations and predictors
n, k = X.shape

# All predictor indices
all_predictors = list(range(1, k))

# Best Subset Selection
best_subset = []
best_subset_Cp = float('inf')
for L in range(1, k):
    for subset in itertools.combinations(all_predictors, L):
        predictors = [0] + list(subset)
        model = sm.OLS(Y, X[:, predictors]).fit()
        current_Cp = Cp(model, sigma2, n)
        if current_Cp < best_subset_Cp:
            best_subset_Cp = current_Cp
            best_subset = predictors

# Forward Selection
included = [0]
best_forward = []
best_forward_Cp = float('inf')
while len(included) < k:
    remaining = list(set(all_predictors) - set(included))
    new_predictor = None
    new_Cp = float('inf')
    for predictor in remaining:
        predictors = included + [predictor]
        model = sm.OLS(Y, X[:, predictors]).fit()
        current_Cp = Cp(model, sigma2, n)
        if current_Cp < new_Cp:
            new_Cp = current_Cp
            new_predictor = predictor
    if new_Cp < best_forward_Cp:
        best_forward_Cp = new_Cp
        best_forward = included + [new_predictor]
        included.append(new_predictor)
    else:
        break

# Backward Selection
included = list(range(k))
best_backward = []
best_backward_Cp = float('inf')
while len(included) > 1:
    best_predictor = None
    worst_Cp = float('-inf')
    for predictor in included[1:]:  
        predictors = list(set(included) - {predictor})
        model = sm.OLS(Y, X[:, predictors]).fit()
        current_Cp = Cp(model, sigma2, n)
        if current_Cp > worst_Cp:
            worst_Cp = current_Cp
            best_predictor = predictor
    if worst_Cp < best_backward_Cp:
        best_backward_Cp = worst_Cp
        best_backward = list(set(included) - {best_predictor})
        included.remove(best_predictor)
    else:
        break

print("The dataset has x1, x2, x3, and y variables.")
print("considering [0,1,2] as [x1, x2, x3]\n")
print("Best Subset Predictors:", best_subset)
print("Forward Selection Predictors:", best_forward)
print("Backward Selection Predictors:", best_backward)


The dataset has x1, x2, x3, and y variables.
considering [0,1,2] as [x1, x2, x3]

Best Subset Predictors: [0, 2]
Forward Selection Predictors: [0, 2]
Backward Selection Predictors: [0]


In [4]:
# Result for best_subset
print("\nBest Subset Selection")
best_subset_model = sm.OLS(Y, X[:, best_subset]).fit()
print("Coefficients:", best_subset_model.params)
print("Residual Variance (σ^2) estimate:", best_subset_model.mse_resid)
print("Standard Errors:", best_subset_model.bse)
print("z-scores:", best_subset_model.tvalues)
print("p-values:", best_subset_model.pvalues)

# Result for forward_selection
print("\nForward Selection")
forward_selection_model = sm.OLS(Y, X[:, best_forward]).fit()
print("Coefficients:", forward_selection_model.params)
print("Residual Variance (σ^2) estimate:", forward_selection_model.mse_resid)
print("Standard Errors:", forward_selection_model.bse)
print("z-scores:", forward_selection_model.tvalues)
print("p-values:", forward_selection_model.pvalues)

# Result for backward_selection
print("\nBackward Selection")
backward_selection_model = sm.OLS(Y, X[:, best_backward]).fit()
print("Coefficients:", backward_selection_model.params)
print("Residual Variance (σ^2) estimate:", backward_selection_model.mse_resid)
print("Standard Errors:", backward_selection_model.bse)
print("z-scores:", backward_selection_model.tvalues)
print("p-values:", backward_selection_model.pvalues)



Best Subset Selection
Coefficients: [-0.19185918  0.04168518]
Residual Variance (σ^2) estimate: 0.1047894582646397
Standard Errors: [0.38342907 0.05166382]
z-scores: [-0.50037726  0.80685433]
p-values: [0.61900486 0.4235719 ]

Forward Selection
Coefficients: [-0.19185918  0.04168518]
Residual Variance (σ^2) estimate: 0.1047894582646397
Standard Errors: [0.38342907 0.05166382]
z-scores: [-0.50037726  0.80685433]
p-values: [0.61900486 0.4235719 ]

Backward Selection
Coefficients: [0.11538462]
Residual Variance (σ^2) estimate: 0.10407239819004521
Standard Errors: [0.04473692]
z-scores: [2.57918086]
p-values: [0.01283039]


### (c)
Use F-test to check whether the model returned from (b) significantly different (α = 0.05) from the complete model in (a)

In [5]:
from scipy.stats import f

def f_test(RSS_reduced, RSS_full, p_reduced, p_full, n, alpha=0.05):
    # Calculate the F-statistic
    F_statistic = ((RSS_reduced - RSS_full) / (p_full - p_reduced)) / (RSS_full / (n - p_full - 1))

    # Degrees of freedom
    df1 = p_full - p_reduced
    df2 = n - p_full - 1
    
    # Calculate the p-value
    p_value = 1 - f.cdf(F_statistic, df1, df2)
    
    # Make a decision
    if p_value < alpha:
        decision = "Rejected"
    else:
        decision = "Failed to Reject"
    
    return F_statistic, p_value, decision

# compare the full model from (a) with the reduced models from (b)

# Reduced model 1
RSS_reduced = backward_selection_model.mse_resid
RSS_full = full_model.mse_resid
p_reduced = len(best_backward)
p_full = len(full_model.params)
n = len(Y)

F_statistic, p_value, decision = f_test(RSS_reduced, RSS_full, p_reduced, p_full, n)
print("F-statistic:", F_statistic)
print("p-value:", p_value)
print("Decision:", decision)

# Reduced model 2
RSS_reduced = forward_selection_model.mse_resid
p_reduced = len(best_forward)
n = len(Y)

F_statistic, p_value, decision = f_test(RSS_reduced, RSS_full, p_reduced, p_full, n)
print("\nF-statistic:", F_statistic)
print("p-value:", p_value)
print("Decision:", decision)

# Reduced model 3
RSS_reduced = best_subset_model.mse_resid
RSS_full = full_model.mse_resid
p_reduced = len(best_subset)
p_full = len(full_model.params)
n = len(Y)

F_statistic, p_value, decision = f_test(RSS_reduced, RSS_full, p_reduced, p_full, n)
print("\nF-statistic:", F_statistic)
print("p-value:", p_value)
print("Decision:", decision)



F-statistic: -0.6099576807085887
p-value: 1.0
Decision: Failed to Reject

F-statistic: -0.8976000159090993
p-value: 1.0
Decision: Failed to Reject

F-statistic: -0.8976000159090993
p-value: 1.0
Decision: Failed to Reject
